In [1]:
import os
import ctypes
from datetime import datetime
from win32file import (
    CreateFile,
    GetFileInformationByHandle,
    FILE_SHARE_READ,
    FILE_SHARE_WRITE,
    FILE_SHARE_DELETE,
    OPEN_EXISTING,
    FILE_FLAG_BACKUP_SEMANTICS,
)
from win32api import GetVolumeInformation  # 修改为从 win32api 导入
from pywintypes import TimeType
import time
import uuid


# 哈希计算函数
def hash_data(data, hash_val):
    for byte in data:
        hash_val ^= ((hash_val * 0x820) + (byte & 0x00000000000000FF)) + (hash_val >> 2)
    return hash_val


# 转换为 FAT 日期时间格式 (DOS 时间)
def time_to_fat(dt):
    # 获取 FAT 日期和 FAT 时间
    fat_date = ((dt.year - 1980) << 9) | ((dt.month) << 5) | dt.day
    fat_time = ((dt.hour) << 11) | ((dt.minute) << 5) | (dt.second // 2)
    return fat_date, fat_time


# 获取系统卷的 CLSID（或者卷 GUID）
def get_system_clsid(drive_path):
    volume_name = drive_path
    volume_guid = ctypes.create_unicode_buffer(1024)

    

    # 获取卷信息
    if GetVolumeInformation(volume_name):
        # 截取返回的卷 GUID
        return volume_guid.value.strip('\\')
    else:
        return None


# 计算文件哈希值的函数
def hash_file(file_path):
    # 初始哈希值
    hash_val = 0x95E729BA2C37FD21

    # 获取系统 CLSID（卷 GUID）
    drive_letter = file_path[0:3]  # 假设文件路径的前3个字符表示盘符，如 "G:/"
    clsid = get_system_clsid(drive_letter)

    if clsid:
        # 哈希 Volume GUID (CLSID)
        clsid_bytes = uuid.UUID(clsid).bytes
        hash_val = hash_data(clsid_bytes, hash_val)
        print(f"Hashing Volume GUID (CLSID): {clsid}")

    # 打开文件句柄
    try:
        file_handle = CreateFile(
            file_path,
            0,
            FILE_SHARE_READ | FILE_SHARE_WRITE | FILE_SHARE_DELETE,
            None,
            OPEN_EXISTING,
            FILE_FLAG_BACKUP_SEMANTICS,
            None,
        )
    except Exception as e:
        print(f"Failed to open file: {e}")
        return

    # 获取文件信息
    file_info = GetFileInformationByHandle(file_handle)
    print("File Information:", file_info)
    # File ID is composed of nFileIndexHigh and nFileIndexLow
    file_id_high = file_info[8]
    file_id_low = file_info[9]

    # Calculate the full file ID
    file_id = (file_id_high << 32) | file_id_low

    # 哈希文件ID
    hash_val = hash_data(file_id.to_bytes(8, byteorder='little'), hash_val)

    # 获取文件扩展名
    file_extension = os.path.splitext(file_path)[1]
    hash_val = hash_data(file_extension.encode('utf-16le'), hash_val)

    # 获取文件最后修改时间 (DOS时间)
    last_write_time = datetime.fromtimestamp(os.path.getmtime(file_path))
    fat_date, fat_time = time_to_fat(last_write_time)
    dos_time = (fat_date << 16) | fat_time

    # 哈希最后修改时间
    hash_val = hash_data(dos_time.to_bytes(4, byteorder='little'), hash_val)

    # 打印计算的哈希值
    print(f"Calculated Hash for file {file_path}: {hash_val:#018x}")

    return hash_val


# 主程序
if __name__ == "__main__":
    file_path = r"G:/图谱/23.08.12双彩虹\Z30_2126-NEF_DxO_DeepPRIMEXD.jpg"  # 文件路径
    hash_file(file_path)

File Information: (32, pywintypes.datetime(2023, 10, 14, 9, 35, 48, 969000, tzinfo=TimeZoneInfo('GMT Standard Time', True)), pywintypes.datetime(2024, 8, 21, 8, 49, 10, 439000, tzinfo=TimeZoneInfo('GMT Standard Time', True)), pywintypes.datetime(2023, 8, 12, 11, 38, 12, 150000, tzinfo=TimeZoneInfo('GMT Standard Time', True)), 1038367535, 0, 4894243, 1, 1441792, 21672)
Calculated Hash for file G:/图谱/23.08.12双彩虹\Z30_2126-NEF_DxO_DeepPRIMEXD.jpg: 0xcc86e45911ce92e7b503ddaad25317bd4ac4afbbcce8505ebb728185989a36d2523b9dc


In [37]:
def get_system_clsid(drive_path):
    volume_name = drive_path
    volume_guid = ctypes.create_unicode_buffer(1024)

    # 获取卷信息
    if GetVolumeInformation(volume_name):
        # 截取返回的卷 GUID
        return volume_guid.value.strip('\\')
    else:
        return None

# 示例用法
drive_path = "G:\\"
clsid = get_system_clsid(drive_path)
print(f"CLSID for drive {drive_path}: {clsid}")

CLSID for drive G:\: 


In [45]:
import wmi


def get_hard_disk_serial_number():
    c = wmi.WMI()
    for physical_disk in c.Win32_DiskDrive():
        serial_number = physical_disk.SerialNumber
        print("硬盘序列号：", serial_number)


get_hard_disk_serial_number()

硬盘序列号： 0000_0000_0000_0000_A428_B701_C852_0082.
硬盘序列号： 001B_448B_4583_B6CF.


In [50]:
# import ctypes
# from ctypes import wintypes
# import comtypes.client


# # Define necessary structures and constants
# class SIZE(ctypes.Structure):
#     _fields_ = [("cx", ctypes.c_long), ("cy", ctypes.c_long)]


# class BITMAP(ctypes.Structure):
#     _fields_ = [
#         ("bmType", ctypes.c_long),
#         ("bmWidth", ctypes.c_long),
#         ("bmHeight", ctypes.c_long),
#         ("bmWidthBytes", ctypes.c_long),
#         ("bmPlanes", ctypes.c_uint16),
#         ("bmBitsPixel", ctypes.c_uint16),
#         ("bmBits", ctypes.c_void_p),
#     ]


# class BITMAPFILEHEADER(ctypes.Structure):
#     _fields_ = [
#         ("bfType", ctypes.c_uint16),
#         ("bfSize", ctypes.c_uint32),
#         ("bfReserved1", ctypes.c_uint16),
#         ("bfReserved2", ctypes.c_uint16),
#         ("bfOffBits", ctypes.c_uint32),
#     ]


# class BITMAPINFOHEADER(ctypes.Structure):
#     _fields_ = [
#         ("biSize", ctypes.c_uint32),
#         ("biWidth", ctypes.c_long),
#         ("biHeight", ctypes.c_long),
#         ("biPlanes", ctypes.c_uint16),
#         ("biBitCount", ctypes.c_uint16),
#         ("biCompression", ctypes.c_uint32),
#         ("biSizeImage", ctypes.c_uint32),
#         ("biXPelsPerMeter", ctypes.c_long),
#         ("biYPelsPerMeter", ctypes.c_long),
#         ("biClrUsed", ctypes.c_uint32),
#         ("biClrImportant", ctypes.c_uint32),
#     ]


# class BITMAPINFO(ctypes.Structure):
#     _fields_ = [("bmiHeader", BITMAPINFOHEADER)]


# BI_RGB = 0
# DIB_RGB_COLORS = 0

# # Load necessary Windows API functions
# gdi32 = ctypes.windll.gdi32
# user32 = ctypes.windll.user32


# def create_shell_item(file_name):
#     shell_item = comtypes.client.CreateObject('Shell.Application').NameSpace(file_name).Self
#     return shell_item


# def get_bmp(file_path, width, height):
#     comtypes.CoInitialize()

#     shell_item = create_shell_item(file_path)
#     factory = shell_item.QueryInterface(comtypes.gen.Shell32.IShellItemImageFactory)

#     size = SIZE(cx=width, cy=height)
#     hbitmap = factory.GetImage(size, 0x08)

#     bmp = BITMAP()
#     gdi32.GetObjectW(hbitmap, ctypes.sizeof(BITMAP), ctypes.byref(bmp))

#     hdc = gdi32.CreateCompatibleDC(None)
#     bitmap_info = BITMAPINFO()
#     bitmap_info.bmiHeader.biSize = ctypes.sizeof(BITMAPINFOHEADER)
#     bitmap_info.bmiHeader.biWidth = bmp.bmWidth
#     bitmap_info.bmiHeader.biHeight = bmp.bmHeight
#     bitmap_info.bmiHeader.biPlanes = 1
#     bitmap_info.bmiHeader.biBitCount = bmp.bmBitsPixel
#     bitmap_info.bmiHeader.biCompression = BI_RGB

#     byte_size = bmp.bmWidthBytes * bmp.bmHeight
#     bits = (ctypes.c_ubyte * byte_size)()

#     gdi32.GetDIBits(hdc, hbitmap, 0, bmp.bmHeight, bits, ctypes.byref(bitmap_info), DIB_RGB_COLORS)

#     gdi32.DeleteDC(hdc)
#     gdi32.DeleteObject(hbitmap)

#     file_header = BITMAPFILEHEADER()
#     file_header.bfType = 0x4D42
#     file_header.bfSize = (
#         ctypes.sizeof(BITMAPFILEHEADER) + ctypes.sizeof(BITMAPINFOHEADER) + byte_size
#     )
#     file_header.bfOffBits = ctypes.sizeof(BITMAPFILEHEADER) + ctypes.sizeof(BITMAPINFOHEADER)

#     return bytes(file_header) + bytes(bitmap_info.bmiHeader) + bytes(bits)


# # Example usage
# bmp_data = get_bmp(r"G:\图谱\23.08.12双彩虹\Z30_2126-NEF_DxO_DeepPRIMEXD.jpg", 96, 96)

COMError: (-2147467259, '未指定的错误', (None, None, None, 0, None))

In [11]:
import ctypes
from ctypes import windll, POINTER, byref, WinError, HRESULT
from ctypes.wintypes import DWORD, LONG, WORD, UINT

import comtypes
from comtypes import GUID, COMMETHOD, IUnknown

# 手动定义缺少的类型
LPCWSTR = ctypes.c_wchar_p
HDC = ctypes.c_void_p


# 定义 HBITMAP 为一个新的类型
class HBITMAP(ctypes.c_void_p):
    pass


# 定义 SIZE 结构
class SIZE(ctypes.Structure):
    _fields_ = [("cx", ctypes.c_long), ("cy", ctypes.c_long)]


# 定义 SIIGBF 枚举
class SIIGBF:
    SIIGBF_RESIZETOFIT = 0
    SIIGBF_BIGGERSIZEOK = 0x00000001
    SIIGBF_MEMORYONLY = 0x00000002
    SIIGBF_ICONONLY = 0x00000004
    SIIGBF_THUMBNAILONLY = 0x00000008
    SIIGBF_INCACHEONLY = 0x00000010


# 手动定义 IShellItemImageFactory 接口
class IShellItemImageFactory(IUnknown):
    _iid_ = GUID('{bcc18b79-ba16-442f-80c4-8a59c30c463b}')
    _methods_ = [
        COMMETHOD(
            [],
            HRESULT,
            'GetImage',
            (['in'], SIZE, 'size'),
            (['in'], ctypes.c_int, 'flags'),
            (['out'], POINTER(HBITMAP), 'phbm'),
        )
    ]


# 定义 IShellItem 接口
class IShellItem(IUnknown):
    _iid_ = GUID('{43826D1E-E718-42EE-BC55-A1E261C37BFE}')
    _methods_ = []


def get_thumbnail(file_path, width, height):
    from comtypes import CoInitialize, CoUninitialize

    CoInitialize()

    # 创建 IShellItem
    SHCreateItemFromParsingName = windll.shell32.SHCreateItemFromParsingName
    SHCreateItemFromParsingName.argtypes = [
        LPCWSTR,
        ctypes.c_void_p,
        POINTER(GUID),
        POINTER(ctypes.c_void_p),
    ]
    SHCreateItemFromParsingName.restype = HRESULT

    shell_item = ctypes.POINTER(IShellItem)()
    hr = SHCreateItemFromParsingName(file_path, None, byref(IShellItem._iid_), byref(shell_item))
    if hr != 0:
        raise WinError(hr)

    # 获取 IShellItemImageFactory 接口
    factory = shell_item.QueryInterface(IShellItemImageFactory)

    # 获取缩略图图像
    size = SIZE(width, height)
    flags = SIIGBF.SIIGBF_BIGGERSIZEOK

    hbitmap = HBITMAP()
    hbitmap = factory.GetImage(size, flags)
    # if hr != 0:
    #     raise WinError(hr)

    if not hbitmap:
        raise WinError("无法获取缩略图")

    # 将 HBITMAP 转换为字节数据
    # 获取 BITMAP 信息
    class BITMAP(ctypes.Structure):
        _fields_ = [
            ('bmType', LONG),
            ('bmWidth', LONG),
            ('bmHeight', LONG),
            ('bmWidthBytes', LONG),
            ('bmPlanes', WORD),
            ('bmBitsPixel', WORD),
            ('bmBits', ctypes.c_void_p),
        ]

    bitmap = BITMAP()
    res = windll.gdi32.GetObjectW(hbitmap, ctypes.sizeof(BITMAP), byref(bitmap))
    if res == 0:
        raise WinError()

    # 获取位图数据
    class BITMAPINFOHEADER(ctypes.Structure):
        _fields_ = [
            ('biSize', DWORD),
            ('biWidth', LONG),
            ('biHeight', LONG),
            ('biPlanes', WORD),
            ('biBitCount', WORD),
            ('biCompression', DWORD),
            ('biSizeImage', DWORD),
            ('biXPelsPerMeter', LONG),
            ('biYPelsPerMeter', LONG),
            ('biClrUsed', DWORD),
            ('biClrImportant', DWORD),
        ]

    class BITMAPINFO(ctypes.Structure):
        _fields_ = [
            ('bmiHeader', BITMAPINFOHEADER),
        ]

    bmi = BITMAPINFO()
    bmi.bmiHeader.biSize = ctypes.sizeof(BITMAPINFOHEADER)
    bmi.bmiHeader.biWidth = bitmap.bmWidth
    bmi.bmiHeader.biHeight = bitmap.bmHeight
    bmi.bmiHeader.biPlanes = 1
    bmi.bmiHeader.biBitCount = bitmap.bmBitsPixel
    bmi.bmiHeader.biCompression = 0  # BI_RGB

    # 计算图像数据大小
    image_size = ((bitmap.bmWidth * bitmap.bmBitsPixel + 31) // 32) * 4 * bitmap.bmHeight
    image_buffer = (ctypes.c_byte * image_size)()

    hdc = windll.gdi32.CreateCompatibleDC(0)
    res = windll.gdi32.GetDIBits(hdc, hbitmap, 0, bitmap.bmHeight, image_buffer, byref(bmi), 0)
    if res == 0:
        raise WinError()

    windll.gdi32.DeleteDC(hdc)
    windll.gdi32.DeleteObject(hbitmap)

    # 构建位图文件头
    class BITMAPFILEHEADER(ctypes.Structure):
        _pack_ = 1
        _fields_ = [
            ('bfType', WORD),
            ('bfSize', DWORD),
            ('bfReserved1', WORD),
            ('bfReserved2', WORD),
            ('bfOffBits', DWORD),
        ]

    file_header_size = ctypes.sizeof(BITMAPFILEHEADER)
    info_header_size = ctypes.sizeof(BITMAPINFOHEADER)
    file_size = file_header_size + info_header_size + image_size

    bmfh = BITMAPFILEHEADER()
    bmfh.bfType = 0x4D42  # 'BM' 的 ASCII 码
    bmfh.bfSize = file_size
    bmfh.bfReserved1 = 0
    bmfh.bfReserved2 = 0
    bmfh.bfOffBits = file_header_size + info_header_size

    # 组合文件头、信息头和图像数据
    import struct

    bmp_data = bytearray()
    bmp_data.extend(
        struct.pack(
            '<HIHHI', bmfh.bfType, bmfh.bfSize, bmfh.bfReserved1, bmfh.bfReserved2, bmfh.bfOffBits
        )
    )
    bmp_data.extend(bytearray(bytes(bmi.bmiHeader)))
    bmp_data.extend(image_buffer)

    CoUninitialize()

    return bmp_data


# 示例使用
file_path = (
    r"G:\图谱\23.08.12双彩虹\Z30_2126-NEF_DxO_DeepPRIMEXD.jpg"  # 请将此路径替换为您的文件路径
)

# 获取缩略图数据
bmp_data = get_thumbnail(file_path, 96, 96)  # 96x96 尺寸


缩略图已保存到 thumbnail.bmp


In [15]:
import os
import time

folder_path = r"G:\图谱\23.03.26海滨栈道+中英街"
temp_folder = os.path.join(folder_path, '.temp')
os.makedirs(temp_folder, exist_ok=True)

image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png'))]

if not image_files:
    print("No image files found in the specified directory.")
else:
    start_time = time.time()
    for i, image_file in enumerate(image_files):
        print(f"Testing file: {image_file}")
        bmp_data = get_thumbnail(image_file, 96, 96)
        output_file = os.path.join(temp_folder, f"thumbnail_{i}.bmp")
        with open(output_file, "wb") as f:
            f.write(bmp_data)
    end_time = time.time()
    avg_time = (end_time - start_time) / len(image_files)
    print(f"Average loading time per image: {avg_time:.4f} seconds")

Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0704.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0708.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0740.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0748.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0761.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0808.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0814.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0817.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0821.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0824.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0831.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0850.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0874.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0879.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0902.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0903.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0977.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0987.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_0991.JPG
Testing file: G:\图谱\23.03.26海滨栈道+中英街\Z30_1030.JPG


In [2]:
import requests
import concurrent.futures
import time

url = "http://localhost:8000/generate_thumbnails"
data = {"folder_path": r"G:/图谱/23.03.26海滨栈道+中英街", "width": 256, "height": 256}

def send_request(worker_id):
    start_time = time.time()
    print(f"Worker {worker_id} started at {start_time}")
    
    response = requests.post(url, json=data)
    
    end_time = time.time()
    print(f"Worker {worker_id} finished at {end_time}")
    
    return response.json()


with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    futures = [executor.submit(send_request, worker_id) for worker_id in range(1)]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)

Worker 0 started at 1734348309.4771912
Worker 0 finished at 1734348312.0101619
{'message': '缩略图生成任务已添加到后台'}


In [1]:
import os
import requests
from PIL import Image
import io
import time

# 创建 .thumbs 文件夹
folder_path = r"G:/图谱/23.03.26海滨栈道+中英街"
thumbs_folder = os.path.join(folder_path, '.thumbs')

os.makedirs(thumbs_folder, exist_ok=True)
url = "http://localhost:8000/get_thumbnail"

# 遍历文件夹中的所有图片
image_files = [
    os.path.join(folder_path, f).replace("\\", '/')
    for f in os.listdir(folder_path)
    if f.lower().endswith(('.jpg', '.png'))
]

start_time = time.time()
total_time = 0

for image_file in image_files:
    file_start_time = time.time()

    # 定义请求的 URL 和 payload
    payload = {"photo_path": image_file, "width": 256, "height": 256}

    # 发送请求
    response = requests.get(url, json=payload)

    # 检查请求是否成功
    if response.status_code == 200:
        # 读取返回的图片
        img = Image.open(io.BytesIO(response.content))

        # 构建输出文件路径
        output_file = os.path.join(
            thumbs_folder,
            os.path.basename(image_file).replace('.JPG', '.webp').replace('.png', '.webp'),
        )

        # 保存图片为 WebP 格式
        img.save(output_file, 'WEBP')
        print(f"Saved thumbnail for {image_file} to {output_file}")
    else:
        print(f"Failed to get thumbnail for {image_file}. Status code: {response.status_code}")

    file_end_time = time.time()
    total_time += file_end_time - file_start_time

end_time = time.time()
avg_time = total_time / len(image_files)
print(f"Total time: {end_time - start_time:.4f} seconds")
print(f"Average time per request: {avg_time:.4f} seconds")

Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0704.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0704.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0708.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0708.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0740.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0740.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0748.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0748.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0761.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0761.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0808.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0808.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0814.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0814.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0817.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0817.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0821.JPG to G:/图谱/23.03.26海滨栈道+中英街\.thumbs\Z30_0821.webp
Saved thumbnail for G:/图谱/23.03.26海滨栈道+中英街/Z30_0824.JPG to G:/图谱

In [6]:
from PIL import Image
from PIL.ExifTags import TAGS
import os
import time

def get_image_capture_time(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if exif_data:
        for tag, value in exif_data.items():
            tag_name = TAGS.get(tag, tag)
            if tag_name == 'DateTimeOriginal':
                return value
    # 如果没有拍摄时间，返回文件的创建时间
    creation_time = time.strftime('%Y:%m:%d %H:%M:%S', time.localtime(os.path.getctime(image_path)))
    return creation_time

image_path = r"G:/图谱/23.03.26海滨栈道+中英街/Z30_0704.JPG"
capture_time = get_image_capture_time(image_path)
print(f"Capture time for {image_path}: {capture_time}")

Capture time for G:/图谱/23.03.26海滨栈道+中英街/Z30_0704.JPG: 2023:07:24 22:48:15


In [1]:
#!/usr/bin/python
# -*- coding:UTF-8 -*-
import os
import sys

path = "/volume1/video_raw/"


def exte(a, *endstring):
    array = map(a.endswith, endstring)
    if True in array:
        return True
    else:
        return False


def make_thumb():
    all_files = []
    for dirpath, dirnames, filenames in os.walk(path):
        for name in filenames:
            if exte(name, ext):
                videopath = os.path.join(dirpath, name)
                rootpath = str(dirpath)
                videoname = str(name)
                print('Found file: ', videopath)
                all_files.append(videopath)
                shell = (
                    'ffmpeg -i %s -y -f mjpeg -ss 3 -t 0.001 -s 1280x720 "%s/@eaDir/%s/SYNOVIDEO_VIDEO_SCREENSHOT.jpg"'
                    % (videopath, rootpath, videoname)
                )
                print(shell)
                os.system(shell)
    print("All files:", path, all_files)
    return print('已生成', type, '文件的缩略图')


exts = ['.mp4', '.avi', '.wmv', '.mkv', '.flv']
for ext in exts:
    make_thumb()

Found file: test.ipynb
Found file: web_api.py
Found file: setting.txt
Found file: log.txt
Found file: LICENSE
Found file: option_train_DistillationIQA.py
Found file: option_train_DistillationIQA_FR.py
Found file: README.md
Found file: test_DistillationIQA.py
Found file: test_DistillationIQA_single.py
Found file: tools.py
Found file: train_DistillationIQA.py
Found file: train_DistillationIQA_FR.py
Found file: config
Found file: description
Found file: FETCH_HEAD
Found file: HEAD
Found file: index
Found file: packed-refs
Found file: applypatch-msg.sample
Found file: commit-msg.sample
Found file: fsmonitor-watchman.sample
Found file: post-update.sample
Found file: pre-applypatch.sample
Found file: pre-commit.sample
Found file: pre-merge-commit.sample
Found file: pre-push.sample
Found file: pre-rebase.sample
Found file: pre-receive.sample
Found file: prepare-commit-msg.sample
Found file: push-to-checkout.sample
Found file: sendemail-validate.sample
Found file: update.sample
Found file: exc

In [ ]:
#!/usr/bin/python
# -*- coding:UTF-8 -*-
##190710
import os
import sys
import subprocess
import argparse

# 定义 ffprobe路径
ffprobe_path = '/volume1/@appstore/ffmpeg/bin/ffprobe'
# 定义 视频文件格式
types = ['.mp4', '.avi', '.wmv', '.mkv', '.flv', '.mov']

# 解析命令行参数
parser = argparse.ArgumentParser()
parser.add_argument(
    "-o",
    "--OverWrite",
    action="store_true",
    help="Overwrite SYNOPHOTO_THUMB_X.jpg files even existed",
)
args = parser.parse_args()

# 获取makethumb.py文件所在的路径
path = os.path.dirname(os.path.abspath(__file__))


# 定义函数SynoPhotoThumb，用于生成缩略图
def SynoPhotoThumb():
    # 遍历指定路径下的所有文件和子目录
    for dirpath, dirnames, filenames in os.walk(path):
        # 跳过@eaDir文件夹
        if '@eaDir' in dirpath:
            dirnames[:] = []
        try:
            # 尝试从文件列表中移除'SYNOPHOTO_FILM_H264.mp4'
            filenames.remove('SYNOPHOTO_FILM_H264.mp4')
            filenames.remove('SYNOPHOTO_FILM_M.mp4')
            filenames.remove('SYNOPHOTO_FILM_MPEG4.mp4')
        except:
            pass
        # 遍历当前目录下的所有文件
        for name in filenames:
            # 获取文件扩展名
            _, ext = os.path.splitext(name)
            # 如果文件的扩展名在types集合中，那么就为这个文件生成缩略图
            if ext.lower() in types:
                videopath = os.path.join(dirpath, name)
                rootpath = str(dirpath)
                videoname = str(name)

                # 删除生成的.fail文件
                shell = 'rm -f "%s/@eaDir/%s/*.fail"' % (rootpath, videoname)
                os.system(shell)

                # 确保缩略图的目录存在，如果不存在要创建，不然ffmpeg会报错，因为ffmpeg不能主动创建目录。
                # 主要用在没被Synology Photos套件处理前，就不会有@eaDir，想要提前生成这些缩略图就要提前创建目录
                thumb_dir = os.path.join(rootpath, '@eaDir', videoname)
                os.makedirs(thumb_dir, exist_ok=True)

                thumb_m_path = os.path.join(rootpath, '@eaDir', videoname, 'SYNOPHOTO_THUMB_M.jpg')
                thumb_xl_path = os.path.join(
                    rootpath, '@eaDir', videoname, 'SYNOPHOTO_THUMB_XL.jpg'
                )

                # 如果发现有已有缩略图错误，可以使用-o 强制覆盖。
                if (not os.path.exists(thumb_m_path)) or args.OverWrite:
                    print('Create SYNOPHOTO_THUMB_M.jpg  of %s' % thumb_m_path)
                    shellM = (
                        'ffmpeg -loglevel warning -ss "$("%s" -loglevel error -of csv=p=0 -show_entries format=duration "%s" | awk \'{print $1/2}\')" -i "%s" -y -vframes 1 -vf scale=-1:480 "%s"'
                        % (ffprobe_path, videopath, videopath, thumb_m_path)
                    )
                    resultM = subprocess.run(
                        shellM, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
                    )
                    if resultM.returncode != 0:
                        print('Error executing command:', resultM.stderr.decode())
                    else:
                        print('Created!', resultM.stdout.decode())

                if (not os.path.exists(thumb_xl_path)) or args.OverWrite:
                    print('Create SYNOPHOTO_THUMB_XL.jpg of %s' % thumb_xl_path)
                    shellXL = (
                        'ffmpeg -loglevel warning -ss "$("%s" -loglevel error -of csv=p=0 -show_entries format=duration "%s" | awk \'{print $1/2}\')" -i "%s" -y -vframes 1 -vf scale=-1:1280 "%s"'
                        % (ffprobe_path, videopath, videopath, thumb_xl_path)
                    )
                    resultXL = subprocess.run(
                        shellXL, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
                    )
                    if resultXL.returncode != 0:
                        print('Error executing command:', resultXL.stderr.decode())
                    else:
                        print('Created!', resultXL.stdout.decode())

                # 检查目录是否为空，如果为空就删除，意味着缩略图创建失败
                if not os.listdir(thumb_dir):
                    os.rmdir(thumb_dir)
    return


SynoPhotoThumb()
print('SynoPhotoThumb_Generater Finished!')

In [4]:
def exte(a, *endstring):
    array = map(a.endswith, endstring)
    if True in array:
        return True
    else:
        return False

exte(
    '/volume1/video_raw/20230605五彩石/漫川关/学校风景/@eaDir/VID_20230603_072707.mp4/SYNOVIDEO_VIDEO_SCREENSHOT.jpg:',
    '.mp4',
)

False

In [3]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('uhd-iqa-metadata.csv')

# 仅保留前两列
df = df.iloc[:, :2]

from sklearn.model_selection import train_test_split

# 将数据随机分割为 0.8:0.2
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 保存为新的 CSV 文件
train_df.to_csv('uhd-iqa-train.csv', index=False)
test_df.to_csv('uhd-iqa-test.csv', index=False)

In [ ]:
import pandas as pd
import os

# 读取 CSV 文件
validation_df = pd.read_csv('dataset/validation.csv')

# 定义图片文件夹路径
image_folder = 'UHD-IQA/challenge/training'

# 检查每一行的图片是否存在，如果不存在则删除该行
validation_df = validation_df[validation_df['image_name'].apply(lambda x: os.path.exists(os.path.join(image_folder, x)))]

# 保存覆盖原文件
validation_df.to_csv('dataset/validation.csv', index=False)

In [7]:
# Read the CSV file
koniq10k_df = pd.read_csv('koniq10k_scores_and_distributions.csv')

# Display the first few rows of the dataframe
# print(koniq10k_df.head())

# Convert the dataframe
koniq10k_df['quality_mos'] = koniq10k_df['MOS_zscore'] / 20
koniq10k_df = koniq10k_df[['image_name', 'quality_mos']]

# Display the first few rows of the modified dataframe
print(koniq10k_df.head())
# Split the data into 80% training and 20% testing
train_koniq10k_df, test_koniq10k_df = train_test_split(koniq10k_df, test_size=0.2, random_state=42)

# Save the split data into new CSV files
train_koniq10k_df.to_csv('koniq10k_train.csv', index=False)
test_koniq10k_df.to_csv('koniq10k_test.csv', index=False)


        image_name  quality_mos
0  10004473376.jpg     3.869181
1  10007357496.jpg     3.436429
2  10007903636.jpg     3.931429
3  10009096245.jpg     3.862188
4    100117038.jpg     3.755625
